# Importing Required Libraries


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import cv2
import seaborn as sns
import glob
import xml.etree.ElementTree as ET
from PIL import Image
import os
import shutil

Importing Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Dataset


The dataset we are using is a mix of ...

Naming structure for the files is as follows,

04-P

xx - n

xx- Image number
n - Uploader (G - Ginnie, P - Pocholo)





## Dataset Preparation

Name directories inside google drive

In [8]:
# Google Drive Version
# label_dir = "/content/drive/MyDrive/IAT481/CV_Proj/labels/"
# images_dir = "/content/drive/MyDrive/IAT481/CV_Proj/images/"

# Local Version
label_dir = "C:\\Users\\Pocholo\\Desktop\\481\\CV_Project\\labels\\"
images_dir = "C:\\Users\\Pocholo\\Desktop\\481\\CV_Project\\images\\"

In [10]:
# I ran this once to clean up file names (Remove label studio unique ID)
#
# for file in os.listdir(label_dir):
#     if file.endswith(".txt"):
#         test = file.split("-")
#         oldName = label_dir + file;
#         newName = label_dir + test[1] + "-" + test[2]
#         os.rename(oldName,newName)
#         # print(file)

# for file in os.listdir(images_dir):
#     if file.endswith(".jpg"):
#         test = file.split("-")
#         oldName = images_dir + file;
#         newName = images_dir + test[1] + "-" + test[2]
#         os.rename(oldName, newName)

for file in os.listdir(images_dir):
    print(file)

for file in os.listdir(label_dir):
    print(file)

01-G.jpg
01-P.jpg
02-G.jpg
02-P.jpg
03-G.jpg
03-P.jpg
04-G.jpg
04-P.jpg
05-G.jpg
05-P.jpg
06-G.jpg
06-P.jpg
07-G.jpg
07-P.jpg
08-G.jpg
08-P.jpg
09-G.jpg
09-P.jpg
10-G.jpg
10-P.jpg
11-G.jpg
11-p.jpg
12-G.jpg
12-p.jpg
13-G.jpg
13-p.jpg
14-G.jpg
14-P.jpg
15-G.jpg
15-P.jpg
16-G.jpg
16-P.jpg
17-G.jpg
17-P.jpg
18-G.jpg
18-P.jpg
19-G.jpg
19-P.jpg
20-G.jpg
20-P.jpg
21-G.jpg
21-P.jpg
22-G.jpg
22-P.jpg
23-G.jpg
23-P.jpg
24-G.jpg
24-P.jpg
25-G.jpg
25-P.jpg
26-G.jpg
26-P.jpg
27-G.jpg
27-P.jpg
28-G.jpg
28-P.jpg
29-G.jpg
29-P.jpg
30-G.jpg
30-P.jpg
01-G.txt
01-P.txt
02-G.txt
02-P.txt
03-G.txt
03-P.txt
04-G.txt
04-P.txt
05-G.txt
05-P.txt
06-G.txt
06-P.txt
07-G.txt
07-P.txt
08-G.txt
08-P.txt
09-G.txt
09-P.txt
10-G.txt
10-P.txt
11-G.txt
11-p.txt
12-G.txt
12-p.txt
13-G.txt
13-p.txt
14-G.txt
14-P.txt
15-G.txt
15-P.txt
16-G.txt
16-P.txt
17-G.txt
17-P.txt
18-G.txt
18-P.txt
19-G.txt
19-P.txt
20-G.txt
20-P.txt
21-G.txt
21-P.txt
22-G.txt
22-P.txt
23-G.txt
23-P.txt
24-G.txt
24-P.txt
25-G.txt
25-P.txt
26-G.txt
2

In [11]:
df = {'name': [],
      'label': [],
      'width': [],
      'height': [],
      'x': [],
      'y': [],
     }

test = {'holder': []}

for file in os.listdir(label_dir):
    if file.endswith(".txt"):
        content = open(label_dir + file)
        readContent = content.readlines()
        for line in readContent:
            clean = line.split(" ")
            df['name'].append(file)
            df['label'].append(clean[0])
            df['x'].append(clean[1])
            df['y'].append(clean[2])
            df['width'].append(clean[3])
            df['height'].append(clean[4])



val = pd.DataFrame(df)
val


,name,label,width,height,x,y
0,01-G.txt,5,0.345558272208639,0.2616136919315404\n,0.5639771801140994,0.2506112469437653
1,01-G.txt,1,0.3162184189079055,0.22982885085574584\n,0.47758761206193967,0.6821515892420539
2,01-P.txt,3,0.11648,0.22200000000000003\n,0.09984,0.505
3,01-P.txt,2,0.13624000000000006,0.26\n,0.23868000000000003,0.496
4,01-P.txt,0,0.10608000000000004,0.1979999999999999\n,0.37439999999999996,0.49700000000000005
...,...,...,...,...,...,...
162,30-P.txt,1,0.4072249589490965,0.3054187192118227\n,0.30213464696223297,0.4273399014778327
163,30-P.txt,1,0.40229885057471243,0.31280788177339924\n,0.6806239737274211,0.5997536945812809
164,30-P.txt,2,0.3054187192118227,0.2524630541871921\n,0.15270935960591134,0.16317733990147779
165,30-P.txt,4,0.36945812807881784,0.28078817733990147\n,0.7380952380952381,0.20443349753694584


Next step is to split the data into training and valuation,

Per instructions we need to use 20/10 split for each dataset, to achieve this I used every picture divisible by 3 (30 images = 10 per person)

In [ ]:
images_dir_train = "C:\\Users\\Pocholo\\Desktop\\481\\CV_Project\\images\\train\\"
images_dir_val = "C:\\Users\\Pocholo\\Desktop\\481\\CV_Project\\images\\val\\"

label_dir_train = "C:\\Users\\Pocholo\\Desktop\\481\\CV_Project\\labels\\train\\"
label_dir_val = "C:\\Users\\Pocholo\\Desktop\\481\\CV_Project\\labels\\val\\"

# for file in os.listdir(images_dir):
#     if file.endswith(".jpg") and int(file.split("-")[0]) % 3 == 0:
#         os.rename(images_dir + file, images_dir_val + file)
#     else:
#         os.rename(images_dir + file, images_dir_train + file)
        
for file in os.listdir(label_dir):
    if file.endswith(".txt") and int(file.split("-")[0]) % 3 == 0:
        os.rename(label_dir + file, label_dir_val + file)
    else:
        os.rename(label_dir + file, label_dir_train + file)

# YOLO Training


In [13]:
import ultralytics
from ultralytics import YOLO

In [14]:
model = YOLO('yolov8s.pt')

100%|██████████| 21.5M/21.5M [00:04<00:00, 4.78MB/s]


In [ ]:
%%writefile config.yaml

# Google Drive
# path: /content/drive/MyDrive/IAT481/CV_Proj/
# train: /content/drive/MyDrive/IAT481/CV_Proj/images/
# val: /content/drive/MyDrive/IAT481/CV_Proj/images/

# Local
path: C:\\Users\\Pocholo\\Desktop\\481\\CV_Project\\
train: C:\\Users\\Pocholo\\Desktop\\481\\CV_Project\\labels\\
val: C:\\Users\\Pocholo\\Desktop\\481\\CV_Project\\labels\\

# Classes
nc: 6

# Class Names
names:
  0: Dime
  1: Loonie
  2: Nickel
  3: Penny
  4: Quarter
  5: Toonie

Overwriting config.yaml


In [ ]:
model.train(data="config.yaml",epochs=5,patience=5,batch=8, lr0=0.0005,imgsz=640)

Ultralytics YOLOv8.1.26 🚀 Python-3.10.12 torch-2.1.0+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=config.yaml, epochs=5, time=None, patience=5, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_widt

100%|██████████| 755k/755k [00:00<00:00, 24.5MB/s]

Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

 12                  -1  1    591360  ultralytics.nn.modules.block.C2f             [768, 256, 1]                 
 13                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 14             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 15                  -1  1    148224  ultralytics.nn.modules.block.C2f             [384, 128, 1]                 
 16                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
 17            [-1, 12]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 18                  -1  1    493056  ultralytics.nn.modules.block.C2f             [384, 256, 1]                 
 19                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
 20             [-1, 9]  1         0  ultralytics.nn.modules.conv.Concat           [1]  

train: Scanning /content/drive/MyDrive/IAT481/CV_Proj/labels... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:42<00:00,  1.18it/s]

...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  25%|██▌       | 1/4 [00:14<00:42, 14.14s/it]

WARNING ⚠️ NMS time limit 2.800s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:51<00:00, 12.75s/it]


                   all         50        139      0.434      0.612      0.503      0.444
                  Dime         50         12      0.328      0.583      0.398      0.339
                Loonie         50         32       0.42      0.781      0.649      0.588
                Nickel         50         18      0.256      0.722      0.347      0.306
                 Penny         50         25       0.52       0.36      0.461      0.407
               Quarter         50         24      0.499      0.583      0.534      0.484
                Toonie         50         28      0.582      0.643       0.63       0.54
Speed: 2.7ms preprocess, 603.9ms inference, 0.0ms loss, 155.1ms postprocess per image
Results saved to runs/detect/train3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7b907397c8e0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
     